# Homework 3 - Optimization
For homework 3, you will apply optimization to a synthetic dataset. This assignment is designed to provide hands-on practices. 

MAKE YOUR OWN COPY OF THIS FILE BEFORE YOU START. 

Complete each task and submit your Jupyter notebook on Blackboard.

# Section:
- Submodular Set Function Optimization
  - Synthetic Dataset
  - Data Visualization
  - Greedy Optimization
  - Grid Coverage Visualization
  - Result Visualization
  - Runtime Analysis

## To-Do Lists
Look out for sections marked "# IMPLEMENT" and "# QUESTION"
- Undergrads: 4 Implement Blocks + 2 Question Block - 6 Points Total
- Masters: 4 Implement Blocks + 2 Question Block - 6 Points Total

Partial credits will be given.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time

: 

## [1] Synthetic Dataset

In [ ]:
grid_dimension = 50 # dimension of square grid on which sensors will be placed
n_sensors = 20 # number of total sensors to choose from
np.random.seed(42) # set radnom seed

In [ ]:
# here we synthetically create a dataset of sensors with random centers and radii
def load_dataset(grid_dimension, n_sensors):
    # sensor candidates
    IDs = list(range(n_sensors))
    df = pd.DataFrame(IDs, columns = ['ID']) 

    df['cr'] = np.random.randint(1, grid_dimension//5, df.shape[0]) # randomly assign radius to each circle
    df['cx'] = np.random.randint(1, grid_dimension-1, df.shape[0]) # randomly assign x coordinate for center of circle
    df['cy'] = np.random.randint(1, grid_dimension-1, df.shape[0]) # randomly assign y coordinate for center of circle
    
    return df

df = load_dataset(grid_dimension, n_sensors)
print(df.shape)
df.head()

## [2] Data Visualization

In [ ]:
# each circle represents a sensor and the area of the circle represents the grid points that sensor covers
df.sort_values(by='cr', ascending=False, inplace=True) # plots bigger circles underneath smaller ones

fig, ax = plt.subplots(figsize=(10,10)) 
ax.set_xticks(list(range(grid_dimension)))
ax.set_yticks(list(range(grid_dimension)))

colors = []
for i in range(df.shape[0]):
    colors.append((np.random.rand(), np.random.rand(), np.random.rand()))

for i,row in df.iterrows():
    c = plt.Circle((row['cx'], row['cy']), row['cr'], color=colors[row['ID']])
    ax.add_artist(c)
    ax.annotate(row['ID'], xy=(row['cx'], row['cy']), ha="center")

## [3] Greedy Optimization

In [ ]:
'''

    Given grid point (x,y), a circle's center (cx, cy), and radius cr.
    
    Output:
    True if grid point (x,y) falls on or within that circle.
    
    def in_circle(x, y, cx, cy, cr):
        return

    -----------------------------------------------------------------------

    Given an nxn grid, circle's center (cx, cy), and radius cr.
    
    Output:
    A set of all grid points that the circle with center (cx, cy) 
    and radius cr covers in a n by n grid.
    
    def covered_pts(n, cx, cy, cr):
        return

    -----------------------------------------------------------------------

    Example code for getting intersection between two sets.
    Think about how you can modify this to get union between two sets.
    
    aset = {1,3,5}
    bset = {1,2,6}
    inter = aset & bset

    -----------------------------------------------------------------------

    Given a set of circle IDs (S) and a new circle ID (c).
    
    Output:
    The union of points covered by S + c.
    
    def F(S, c, df, coveredset=None):
        return

    -----------------------------------------------------------------------

    Maximize coverage of grid points covered given a budget of k sensors.

    Start with an empty set. Determine best circle to add among unchosen circles, 
    add to set and repeat k times for each k, keep track of how many total grid points 
    are covered.

    Output:
    A pair of ordered list.
      - the first ordered list consists of the IDs of chosen circles/sensors at each iteration of greedy
      - the second ordered list consists of the corresponding number of covered grid points 
        after choosing i circles/sensors 
    
    def greedy(df, k):
        return

'''
    
def in_circle(x, y, cx, cy, cr):
    if((x-cx)**2 + (y-cy)**2 <= cr**2):
        return True
    else:
        return False
    
def covered_pts(n, cx, cy, cr):
    xperim = np.arange(cx - cr - 1, cx + cr + 1, dtype=int)
    yperim = np.arange(cy - cr - 1, cy + cr + 1, dtype=int)
    x, y = np.where((xperim[:,np.newaxis] - cx)**2 + (yperim - cy)**2 <= cr**2)
    covered = set()
    for x, y in zip(xperim[x], yperim[y]):
        if((x>0)&(y>0)&(x<=n)&(y<=n)):
            covered.add((x,y))
    return covered
    
def F(S, c, df, coveredset=None):
    for circle in S:
        curCr = df.loc[circle, 'cr']
        curCx = df.loc[circle, 'cx']
        curCy = df.loc[circle, 'cy']
        xperim = np.arange(curCx - curCr - 1, curCx + curCr + 1, dtype=int)
        yperim = np.arange(curCy - curCr - 1, curCy + curCr + 1, dtype=int)
        x, y = np.where((xperim[:,np.newaxis] - curCx)**2 + (yperim - curCy)**2 <= curCr**2)
        for x, y in zip(xperim[x], yperim[y]):
            coveredset.add((x,y))
    cCr = df.loc[c, 'cr']
    cCx = df.loc[c, 'cx']
    cCy = df.loc[c, 'cy']
    xperim = np.arange(cCx - cCr - 1, cCx + cCr + 1, dtype=int)
    yperim = np.arange(cCy - cCr - 1, cCy + cCr + 1, dtype=int)
    x, y = np.where((xperim[:,np.newaxis] - cCx)**2 + (yperim - cCy)**2 <= cCr**2)
    for x, y in zip(xperim[x], yperim[y]):
        coveredset.add((x,y))
    return coveredset
    
def greedy(df, k):
    currentMax = 0
    setCircles = set()
    setPoints = set() #covered
    for circle in df:
        thisCircleCovers = F(setCircles, circle, df, setPoints)
        if(len(thisCircleCovers) > currentMax):
            setPoints = thisCircleCovers
            setCircles.add(circle)
            currentMax = len(thisCircleCovers)
    #once initial optimal circle is found
    for i in range(k-2):   
        currentMax = 0
        prevCircle = df[:,0]
        for circle in df:
            marginalCoverage = len(F(setCircles, circle, df, setPoints)) - len(setPoints)
            if(marginalCoverage > currentMax):
                setPoints = F(setCircles, circle, df, setPoints)
                setCircles.add(circle)
                currentMax = marginalCoverage
                if(currentMax != 0):
                    setCircles.remove(prevCircle)
                prevCircle = circle

    return

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------


In [ ]:
k=10 # budget on how many sensors we are allowed to choose
df['gridpt_set'] = df.apply(lambda row: covered_pts(grid_dimension,row['cx'],row['cy'],row['cr']),axis=1) # add column containing grid points covered by each circle
result = greedy(df, k) # run greedy optimization, should return list of chosen circle IDs where len(result) == k
print(result[0]) # ID
print(result[1]) # #covered

## [4] Grid Coverage Visualization

In [ ]:
'''
    Use your greedy results.

    Show a line plot - 1 to k vs number of grid point covered. 

'''

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------


In [ ]:
'''

    Use your greedy results.

    Show a line plot - 2 to k vs change of number of grid point covered.

    Hint:
    Consecutive difference between list elements.

'''

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------


In [ ]:
'''

Q: If we have an increase #covered of 100 requirement, we will stop at which k?

'''

# -------------------------------------------------------------------------
# QUESTION - 1 Point
# -------------------------------------------------------------------------

# Your answer.


## [5] Result Visualization

In [ ]:
# left figure is a visualization of how many grid points our selection of sensors covers
# we can compare it with the visualization of all sensors from before (right figure)
fig, ax = plt.subplots(ncols=2, figsize=(20,10)) # note we must use plt.subplots, not plt.subplot

ax[0].set_xticks(list(range(grid_dimension)))
ax[0].set_yticks(list(range(grid_dimension)))
ax[1].set_xticks(list(range(grid_dimension)))
ax[1].set_yticks(list(range(grid_dimension)))

b=10
sensors10 = result[0][:b]
ax[0].set_title('k = ' + str(b))
ax[1].set_title('all')

for i,row in df.iterrows():
    c = plt.Circle((row['cx'], row['cy']), row['cr'], color=colors[row['ID']])
    if row['ID'] in sensors10:
        ax[0].add_artist(c)
        ax[0].annotate(row['ID'], xy=(row['cx'], row['cy']), ha="center")
    c = plt.Circle((row['cx'], row['cy']), row['cr'], color=colors[row['ID']])
    ax[1].add_artist(c)
    ax[1].annotate(row['ID'], xy=(row['cx'], row['cy']), ha="center")

## [6] Runtime Analysis

In [ ]:
'''

    Rerun greedy optimization.
    
    Get runtimes for each i in k_arr (for each i, consider n_sensors == k == k_arr[i]).

    Show a line plot - k vs. runtime (sec).

    Hint:
    Use time.time() .

'''

# -------------------------------------------------------------------------
# IMPLEMENT - 1 Point
# -------------------------------------------------------------------------

grid_dimension = 100
k_arr = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


In [ ]:
'''

Q: If we increase grid_dimension from 100 to 500, how much will runtimes increase?

'''

grid_dimension = 500
k_arr = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

# Your answer.
